In [ ]:
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import ticker as ticker
import pandas as pd
from scipy import linalg as la

import gcChebyshev

import ipywidgets

# Model loading

In [ ]:
# submitted model
new_model = tf.keras.models.load_model('gcChebyshev.h5', custom_objects={'gcChebyshev': gcChebyshev.gcChebyshev})

In [ ]:
new_model.summary()

# Data loading

In [ ]:
loadPath = '../Variationsanalyse/data/doe_1/run__0{:04d}/m_1/assembly_6.csv'
numberOfTests = 1400
n = 188559
df = np.zeros((numberOfTests, n, 3), dtype=np.float32)
for testId in range(0, numberOfTests):
    df_temp = pd.read_csv(loadPath.format(testId+1), sep=' ', header=0, dtype=np.float32)
    df[testId,:] = np.reshape(df_temp.values, (-1,3))
    
print(df.shape)

In our experiments we used two data sets: a data set sampled at the corners/edges of the hypercube spanned by the labels, and an data set sampled uniform within the hypercube. The data set sampled at the corners/edges is used to train the neural network, while the uniform sampled data set is used for the following analysis. Unfortunately, the labels of both data sets are not equal, i.e. the offset of the left buffer is stored at the first entry, while the neural network expects it at the last entry. Thus, loading the labels involves a mapping.

In [ ]:
# mapping from sto_2 to doe_1
# ESP_L, ESP_R, SB_L, SN_X, SN_Y, SB_R
# to
# SN_X, SN_Y, SB_L, SB_R, ESP_R, ESP_L
mapping = (3, 4, 2, 5, 1, 0)

labels = np.loadtxt('../Variationsanalyse/data/doe_1/run_matrix_DOE_1_updated_ESP.csv', delimiter=';', usecols=mapping, dtype=np.float32)

print(labels.shape)

# Creating a noisy version

In [ ]:
dfNoise = df + np.random.uniform(-1, 1, df.shape)
print('raw')
print('X_max:', df[...,0].min(), df[...,0].max())
print('Y_max:', df[...,1].min(), df[...,1].max())
print('Z_max:', df[...,2].min(), df[...,2].max())
print('noisy')
print('X_max:', dfNoise[...,0].min(), dfNoise[...,0].max())
print('Y_max:', dfNoise[...,1].min(), dfNoise[...,1].max())
print('Z_max:', dfNoise[...,2].min(), dfNoise[...,2].max())

In [ ]:
plt.figure(figsize=(18,6))
plt.subplot(2,1,1)
plt.title('Sample')
plt.plot(df[2,...])
plt.legend(['X', 'Y', 'Z'])
plt.subplot(2,1,2)
plt.title('Sample with noise')
plt.plot(dfNoise[2,...])
plt.legend(['X', 'Y', 'Z'])
plt.show()

# Generate predicted labels

In [ ]:
predLabels = np.empty_like(labels)
for r in range(0,df.shape[0],100):
    predLabels[r:r+100,...] = new_model(df[r:r+100,...])
    
predLabelsNoise = np.empty_like(labels)
for r in range(0,df.shape[0],100):
    predLabelsNoise[r:r+100,...] = new_model(dfNoise[r:r+100,...])

# Interactive exploration of the results

In [ ]:
@ipywidgets.interact_manual(idx=(0,len(df)-1, 1))
def showReconstructed(idx=0):
    reconstructed = predLabels[idx,...]
    reconstructedNoise = predLabelsNoise[idx,...]
    
    print(np.abs(labels[idx] - reconstructed))
    
    plt.figure(figsize=(18,9))
    
    plt.subplot(3,1,1)
    plt.plot(labels[idx], 'k', label='input')
    plt.plot(reconstructed, 'b', label='reconstructed')
    plt.plot(np.abs(labels[idx] - reconstructed), 'r', label='error')
    plt.legend()
    plt.xticks(np.arange(6), ['SN_X', 'SN_Y', 'SB_L', 'SB_R', 'ESP_R', 'ESP_L'])
    
    plt.subplot(3,1,2)
    plt.plot(labels[idx], 'k', label='input')
    plt.plot(reconstructedNoise, 'b', label='reconstructed')
    plt.plot(np.abs(labels[idx] - reconstructedNoise), 'r', label='error')
    plt.legend()
    plt.xticks(np.arange(6), ['SN_X', 'SN_Y', 'SB_L', 'SB_R', 'ESP_R', 'ESP_L'])
    
    plt.subplot(3,1,3)
    plt.title('Difference betwen noisy and non-noisy prediction')
    plt.bar(np.arange(6), reconstructed - reconstructedNoise)
    plt.xticks(np.arange(6), ['SN_X', 'SN_Y', 'SB_L', 'SB_R', 'ESP_R', 'ESP_L'])
    plt.show()

# Summary of the performance

In [ ]:
res = new_model.evaluate(df, labels, 12)
print("test loss, test acc:", res)

In [ ]:
res = new_model.evaluate(dfNoise, labels, 12)
print("test loss, test acc:", res)

## Precission goal reached

In [ ]:
diffLabels = np.zeros((labels.shape[0], labels.shape[1]-1))
# calculate the hinge error as a length of a vector
diffLabels[:,0] = np.sqrt( (predLabels[:,0] - labels[:,0])**2 + (predLabels[:,1] - labels[:,1])**2)
# the resulting errors are linear
diffLabels[:,1:] = predLabels[:,2:] - labels[:,2:]
diffLabelsNoise = np.zeros((labels.shape[0], labels.shape[1]-1))
# calculate the hinge error as a length of a vector
diffLabelsNoise[:,0] = np.sqrt( (predLabelsNoise[:,0] - labels[:,0])**2 + (predLabelsNoise[:,1] - labels[:,1])**2)
# the resulting errors are linear
diffLabelsNoise[:,1:] = predLabelsNoise[:,2:] - labels[:,2:]

precission = np.linspace(0.0, 0.1, 100)
percentage = np.zeros_like(precission)
percentageNoise = np.zeros_like(precission)

reached = np.all(diffLabels <= 0.05, axis=1)
per = np.count_nonzero(reached) / reached.shape[0] * 100
print('0.05 mm :', per, '%')

for idx, p in enumerate(precission):
    reached = np.all(diffLabels <= p, axis=1)
    percentage[idx] = np.count_nonzero(reached) / reached.shape[0] * 100
    reachedNoise = np.all(diffLabelsNoise <= p, axis=1)
    percentageNoise[idx] = np.count_nonzero(reachedNoise) / reached.shape[0] * 100

plt.figure(figsize=(12,8))
plt.plot(precission, percentage, color='tab:blue')
plt.plot(precission, percentageNoise, color='tab:red')
plt.xticks(np.linspace(0,0.1, 7))
plt.xlim(0, 0.1)
plt.ylim(0, 105)
plt.xlabel('$p_c$')
plt.ylabel('$\\alpha$')
ax = plt.gca()
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=100))
ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.02f mm'))

plt.legend(['$\\bar{A}^u$ without noise', '$\\bar{A}^{u*}$with noise'], frameon=True)

plt.show()

## Worst prediction for each boundary

In [ ]:
diffLabels = np.abs(labels - predLabels)

maximas = np.argmax(diffLabels, axis=0)
minimas = np.argmin(diffLabels, axis=0)

for idx in range(len(maximas)):
    ma = maximas[idx]
    mi = minimas[idx]
    
    legend = ['SN_X', 'SN_Y', 'SB_L', 'SB_R', 'ESP_R', 'ESP_L']
    
    plt.figure(figsize=(18,6))
    plt.suptitle(legend[idx])
    ax1 = plt.subplot(1,2,1)
    plt.title('Maximal error')
    plt.plot(labels[ma], '+k', label='input')
    plt.plot(predLabels[ma,...], '*b', label='reconstructed')
    plt.bar(np.arange(6), labels[ma] - predLabels[ma,...], width=0.25, color='red', label='error')
    plt.grid()
    plt.legend()
    plt.xticks(np.arange(6), legend)
    ax1.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.02f mm'))
    plt.ylim(-1.5, 1.5)
    
    ax2 = plt.subplot(1,2,2)
    plt.title('Minimal error')
    plt.plot(labels[mi], '+k', label='input')
    plt.plot(predLabels[mi,...], '*b', label='reconstructed')
    plt.bar(np.arange(6), labels[mi] - predLabels[mi,...], width=0.25, color='red', label='error')
    plt.legend()
    plt.grid()
    plt.xticks(np.arange(6), legend)
    ax2.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.02f mm'))
    plt.ylim(-1.5, 1.5)

    plt.show()

# Execution times
Mean execution time of 100 consecutive evaluations of the same sample.

In [ ]:
executionTimes = []
for i in range(100):
    start = time.time()
    new_model(df[:1,...])
    end = time.time()
    executionTimes.append(end - start)
    
t_m = np.mean(executionTimes)
print('t:', t_m, 's')

plt.figure(figsize=(18,6))
ax = plt.gca()
plt.plot(executionTimes)
plt.ylabel('t')
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.02f s'))
plt.axhline(t_m, color='k', linestyle='dotted')
plt.show()

Execution times for evaluating 100 samples at once for the whole dataset.

In [ ]:
executionTimes = []
for r in range(0,df.shape[0],100):
    start = time.time()
    new_model(df[r:r+100,...])
    executionTimes.append( time.time() - start )
    
t_m = np.mean(executionTimes)
print('t:', t_m, 's')

plt.figure(figsize=(18,6))
ax = plt.gca()
plt.title('Dataset without noise')
plt.plot(executionTimes)
plt.ylabel('t')
plt.axhline(t_m, color='k', linestyle='dotted')
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.02f s'))
plt.grid()
plt.show()
    
for r in range(0,df.shape[0],100):
    start = time.time()
    new_model(dfNoise[r:r+100,...])
    executionTimes.append( time.time() - start )
    
t_m = np.mean(executionTimes)
print('t:', t_m, 's')

plt.figure(figsize=(18,6))
ax = plt.gca()
plt.title('Dataset with noise')
plt.plot(executionTimes)
plt.ylabel('s')
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.02f s'))
plt.axhline(t_m, color='k', linestyle='dotted')
plt.grid()
plt.show()

# Error Analysis
## Predicted vs. Actual

In [ ]:
diffLabels = predLabels - labels

_, minmax, mean, variance, skewness, kurtosis = stats.describe(diffLabels)

with np.printoptions(precision=3):
    print('min:', minmax[0])
    print('max:', minmax[1])
    print('mean:', mean)
    print('variance:', variance)
    print('skewness:', skewness)
    print('kurtosis', kurtosis)

plt.figure(figsize=(18,3))
plt.boxplot(diffLabels, medianprops=dict(color='tab:blue'), flierprops=dict(markersize=5, marker='.', alpha=0.5, markeredgecolor='tab:blue', markerfacecolor='tab:blue'))

plt.xticks(np.arange(1,7), ['HINGE\_X', 'HINGE\_Y', 'LOCK\_L', 'LOCK\_R', 'BUFFER\_R', 'BUFFER\_L'])
plt.ylabel('$e^u$')
ax = plt.gca()
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
plt.grid()

plt.show()

## Noisy vs. Actual

In [ ]:
diffLabels = predLabelsNoise - labels

_, minmax, mean, variance, skewness, kurtosis = stats.describe(diffLabels)

with np.printoptions(precision=3):
    print('min:', minmax[0])
    print('max:', minmax[1])
    print('mean:', mean)
    print('variance:', variance)
    print('skewness:', skewness)
    print('kurtosis', kurtosis)

plt.figure(figsize=(18,6))
plt.boxplot(diffLabels, medianprops=dict(color='tab:blue'), flierprops=dict(markersize=5, marker='.', alpha=0.5, markeredgecolor='tab:blue', markerfacecolor='tab:blue'))

plt.xticks(np.arange(1,7), ['HINGE\_X', 'HINGE\_Y', 'LOCK\_L', 'LOCK\_R', 'BUFFER\_R', 'BUFFER\_L'])
plt.ylabel('error / mm')
ax = plt.gca()
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
plt.grid()

plt.tight_layout()

plt.show()

## Noisy vs Predicted

In [ ]:
diffLabels = predLabelsNoise - predLabels

_, minmax, mean, variance, skewness, kurtosis = stats.describe(diffLabels)
derivation = np.std(diffLabels, axis=0)

with np.printoptions(precision=3):
    print('min:', minmax[0])
    print('max:', minmax[1])
    print('mean:', mean)
    print('variance:', variance)
    print('skewness:', skewness)
    print('kurtosis', kurtosis)
    print('lower quartile:', 0.6745*derivation)
    print('whiskers:', 2.698*derivation)

plt.figure(figsize=(18,6))
ax = plt.gca()
ax.boxplot(diffLabels, medianprops=dict(color='tab:blue'), flierprops=dict(markersize=5, marker='.', alpha=0.5, markeredgecolor='tab:blue', markerfacecolor='tab:blue'))

plt.xticks(np.arange(1,7), ['HINGE\_X', 'HINGE\_Y', 'LOCK\_L', 'LOCK\_R', 'BUFFER\_R', 'BUFFER\_L'])
ax.set_ylabel('$\\tilde{e}$')
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
plt.grid()

plt.tight_layout()

plt.show()

## Hinge

In [ ]:
diffLabels = predLabels - labels

discritizedXMax = np.zeros((20,))
discritizedXVar = np.zeros((20,))
discritizedXMean = np.zeros((20,))
discritizedXVarTemp = [[] for i in range(20)]
discritizedYVarTemp = [[] for i in range(20)]
discritizedYMax = np.zeros((20,))
for i in range(1400):
    targetX = np.floor(10 * labels[i,0]).astype(np.int64)
    discritizedXMax[targetX+10] = max( discritizedXMax[targetX+10], abs(diffLabels[i,0]) )
    discritizedXVarTemp[targetX+10].append(abs(diffLabels[i,0]))
    
    targetY = np.floor(10 * labels[i,1]).astype(np.int64)
    discritizedYMax[targetY+10] = max( discritizedYMax[targetY+10], abs(diffLabels[i,1]) )
    discritizedYVarTemp[targetY+10].append(abs(diffLabels[i,1]))

In [ ]:
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.01

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.2, height]

fig = plt.figure(figsize=(18,8))

ax_scatter = plt.axes(rect_scatter)
ax_scatter.tick_params(direction='in', top=True, right=True)
ax_histx = plt.axes(rect_histx)
ax_histx.tick_params(direction='in', labelbottom=False)
ax_histy = plt.axes(rect_histy)
ax_histy.tick_params(direction='in', labelleft=False)

magnitudeDiff = np.sqrt(diffLabels[:,0]**2 + diffLabels[:,1]**2)

# the scatter plot:
sca = ax_scatter.scatter(labels[:,0], labels[:,1], s=magnitudeDiff*2e2, alpha=0.8, edgecolors='none', color='tab:blue')

ax_histx.bar(np.arange(20)+0.5, discritizedXMax)

ax_histy.barh(np.arange(20)+0.5, discritizedYMax)

ax_scatter.set_xlabel('HINGE\_X')
ax_scatter.set_xticks(np.arange(-1, 1.5, 0.5))
ax_scatter.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))

ax_scatter.set_ylabel('HINGE\_Y')
ax_scatter.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))
ax_scatter.grid()

ax_histy.set_xlabel('$e_H^u|_y$')
ax_histy.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
#ax_histy.set_xticks(np.array([0.0, 25.0, 50.0])*1e-3)
plt.setp( ax_histy.xaxis.get_majorticklabels(), ha="left" )
ax_histy.grid()

ax_histx.set_ylabel('$e_H^u|_x$')
ax_histx.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
#ax_histx.set_yticks(np.array([0.0, 25.0, 50.0])*1e-3)
ax_histx.grid()

ax_histy.set_ylim(-1, 21)
ax_histx.set_xlim(-1, 21)

handles, l = sca.legend_elements(
    prop="sizes",
    num=5,
    color='tab:blue',
    markeredgewidth=0,
    alpha=0.8,
    fmt="{x:.0f}$\\mu m$",
    func=lambda s: s/2*10
)
#l = ['$\\mathdefault{0.03}$', '$\\mathdefault{0.06}$', '$\\mathdefault{0.09}$', '$\\mathdefault{0.12}$', '$\\mathdefault{0.15}$', '$\\mathdefault{0.18}$', '$\\mathdefault{0.21}$', '$\\mathdefault{0.24}$']
legend2 = ax_scatter.legend(
    handles, 
    l, 
    loc="upper right", 
    title="Magnitude", 
    bbox_to_anchor=(1.335, 1.345), 
    ncol=2,
    fontsize=5,
    labelspacing=0.5,
    frameon=True,
    fancybox=False,
    handletextpad=0.,
    columnspacing=0.5
)

plt.show()

## Lock

In [ ]:
plt.figure(figsize=(18,6))

ax1 = plt.subplot(2,1,1)
scatterLeft = ax1.scatter(labels[:,2], diffLabels[:,2], s=3, c=labels[:,0], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax1.set_ylabel('$e_{Ll}^u$')
ax1.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax1.grid()

plt.setp(ax1.get_xticklabels(), visible=False)

cbar1 = plt.colorbar(scatterLeft, label=r'$HINGE\_X$')
cbar1.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))


ax3 = plt.subplot(2,1,2, sharex=ax1)
scatterLeftBottom = ax3.scatter(labels[:,2], diffLabels[:,2], s=3, c=labels[:,5], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax3.set_xlabel(r'$LOCK\_L$')
ax3.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))
ax3.grid()

ax3.set_ylabel('$e_{Ll}^u$')
ax3.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))

cbar3 = plt.colorbar(scatterLeftBottom, label=r'$BUFFER\_L$')
cbar3.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

plt.subplots_adjust(hspace=0.05)

plt.show()

plt.figure(figsize=(18,6))

ax2 = plt.subplot(2,1,1)
scatterRight = plt.scatter(labels[:,3], diffLabels[:,3], s=3, c=labels[:,0], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax2.set_ylabel('$e_{Lr}^u$')
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax2.grid()

plt.setp(ax2.get_xticklabels(), visible=False)

cbar2 = plt.colorbar(scatterRight, label=r'$HINGE\_Y$')
cbar2.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

ax4 = plt.subplot(2,1,2, sharex=ax2)
scatterRight = plt.scatter(labels[:,3], diffLabels[:,3], s=3, c=labels[:,4], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax4.set_xlabel(r'$LOCK\_R$')
ax4.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))

ax4.set_ylabel('$e_{Lr}^u$')
ax4.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax4.grid()

cbar4 = plt.colorbar(scatterRight, label=r'$BUFFER\_R$')
cbar4.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

plt.subplots_adjust(hspace=0.05)

plt.show()

In [ ]:
diffLabels[:,3].max()

## Buffers

In [ ]:
plt.figure(figsize=(18,6))

ax1 = plt.subplot(2,1,1)
scatterLeft = ax1.scatter(labels[:,5], diffLabels[:,5], s=1, c=labels[:,0], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax1.set_ylabel('$e_{Bl}^u$')
ax1.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax1.grid()

plt.setp(ax1.get_xticklabels(), visible=False)

cbar1 = plt.colorbar(scatterLeft, label=r'$HINGE\_X$')
cbar1.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))


ax3 = plt.subplot(2,1,2, sharex=ax1)
scatterLeftBottom = ax3.scatter(labels[:,5], diffLabels[:,5], s=1, c=labels[:,2], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax3.set_xlabel(r'$BUFFER\_L$')
ax3.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))

ax3.set_ylabel('$e_{Bl}^u$')
ax3.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax3.grid()

cbar3 = plt.colorbar(scatterLeftBottom, label=r'$LOCK\_L$')
cbar3.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

plt.subplots_adjust(hspace=0.05)

plt.show()

plt.figure(figsize=(18,6))

ax2 = plt.subplot(2,1,1)
scatterRight = plt.scatter(labels[:,4], diffLabels[:,4], s=1, c=labels[:,1], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax2.set_ylabel('$e_{Br}^u$')
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax2.grid()

plt.setp(ax2.get_xticklabels(), visible=False)

cbar2 = plt.colorbar(scatterRight, label=r'$HINGE\_Y$')
cbar2.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

ax4 = plt.subplot(2,1,2, sharex=ax2)
scatterRight = plt.scatter(labels[:,4], diffLabels[:,4], s=1, c=labels[:,3], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax4.set_xlabel('$BUFFER\_R$')
ax4.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))

ax4.set_ylabel('$e_{Br}^u$')
ax4.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax4.grid()

cbar4 = plt.colorbar(scatterRight, label=r'$LOCK\_R$')
cbar4.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

plt.subplots_adjust(hspace=0.05)

plt.show()

In [ ]:
print(diffLabels[:,4].min(), diffLabels[:,4].max())
print(diffLabels[:,5].min(), diffLabels[:,5].max())

# Noisy data

In [ ]:
diffLabels = predLabelsNoise - labels

discritizedX = np.zeros((20,))
discritizedY = np.zeros((20,))
for i in range(1400):
    targetX = np.floor(10 * labels[i,0]).astype(np.int64)
    discritizedX[targetX+10] = max( discritizedX[targetX+10], abs(diffLabels[i,0]) )
    
    targetY = np.floor(10 * labels[i,1]).astype(np.int64)
    discritizedY[targetY+10] = max( discritizedY[targetY+10], abs(diffLabels[i,1]) )

In [ ]:
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.01


rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.2, height]

fig = plt.figure(figsize=(18,8))

ax_scatter = plt.axes(rect_scatter)
ax_scatter.tick_params(direction='in', top=True, right=True)
ax_histx = plt.axes(rect_histx)
ax_histx.tick_params(direction='in', labelbottom=False)
ax_histy = plt.axes(rect_histy)
ax_histy.tick_params(direction='in', labelleft=False)

magnitudeDiff = np.sqrt(diffLabels[:,0]**2 + diffLabels[:,1]**2)

# the scatter plot:
ax_scatter.scatter(labels[:,0], labels[:,1], s=magnitudeDiff*2e2, alpha=0.8, edgecolors='none', color='tab:blue')

ax_histx.bar(np.arange(20)+0.5, discritizedX)
ax_histy.barh(np.arange(20)+0.5, discritizedY)


ax_scatter.set_xlabel('HINGE\_X')
ax_scatter.set_xticks(np.arange(-1, 1.5, 0.5))
ax_scatter.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))

ax_scatter.set_ylabel('HINGE\_Y')
ax_scatter.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))
ax_scatter.grid()

ax_histy.set_xlabel('$e_H^u|_y$')
ax_histy.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
plt.setp( ax_histy.xaxis.get_majorticklabels(), ha="left" )
ax_histy.grid()

ax_histx.set_ylabel('$e_H^u|_x$')
ax_histx.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax_histx.grid()

ax_histy.set_ylim(-1, 21)
ax_histx.set_xlim(-1, 21)

handles, l = sca.legend_elements(
    prop="sizes",
    num=5,
    color='tab:blue',
    markeredgewidth=0,
    alpha=0.8,
    fmt="{x:.0f}$\\mu m$",
    func=lambda s: s/2*10
)
legend2 = ax_scatter.legend(
    handles, 
    l, 
    loc="upper right", 
    title="Magnitude", 
    bbox_to_anchor=(1.335, 1.345), 
    ncol=2,
    fontsize=5,
    labelspacing=0.5,
    frameon=True,
    fancybox=False,
    handletextpad=0.,
    columnspacing=0.5
)

plt.show()

## Lock

In [ ]:
plt.figure(figsize=(18,6))

ax1 = plt.subplot(2,1,1)
scatterLeft = ax1.scatter(labels[:,2], diffLabels[:,2], s=3, c=labels[:,0], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax1.set_ylabel('$e_{Ll}^u$')
ax1.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax1.grid()

plt.setp(ax1.get_xticklabels(), visible=False)

cbar1 = plt.colorbar(scatterLeft, label=r'$HINGE\_X$')
cbar1.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

ax3 = plt.subplot(2,1,2, sharex=ax1)
scatterLeftBottom = ax3.scatter(labels[:,2], diffLabels[:,2], s=3, c=labels[:,5], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax3.set_xlabel(r'$LOCK\_L$')
ax3.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))

ax3.set_ylabel('$e_{Ll}^u$')
ax3.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax3.grid()

cbar3 = plt.colorbar(scatterLeftBottom, label=r'$BUFFER\_L$')
cbar3.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

plt.subplots_adjust(hspace=0.05)

plt.show()

plt.figure(figsize=(18,6))

ax2 = plt.subplot(2,1,1)
scatterRight = plt.scatter(labels[:,3], diffLabels[:,3], s=3, c=labels[:,0], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax2.set_ylabel('$e_{Lr}^u$')
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax2.grid()

plt.setp(ax2.get_xticklabels(), visible=False)

cbar2 = plt.colorbar(scatterRight, label=r'$HINGE\_Y$')
cbar2.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

ax4 = plt.subplot(2,1,2, sharex=ax2)
scatterRight = plt.scatter(labels[:,3], diffLabels[:,3], s=3, c=labels[:,4], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax4.set_xlabel(r'$LOCK\_R$')
ax4.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))

ax4.set_ylabel('$e_{Lr}^u$')
ax4.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax4.grid()

cbar4 = plt.colorbar(scatterRight, label=r'$BUFFER\_R$')
cbar4.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

plt.subplots_adjust(hspace=0.05)

plt.show()

In [ ]:
diffLabels[:,3].max()

## Buffers

In [ ]:
plt.figure(figsize=(18,6))

ax1 = plt.subplot(2,1,1)
scatterLeft = ax1.scatter(labels[:,5], diffLabels[:,5], s=3, c=labels[:,0], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax1.set_ylabel('$e_{Bl}^u$')
ax1.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax1.grid()

plt.setp(ax1.get_xticklabels(), visible=False)

cbar1 = plt.colorbar(scatterLeft, label=r'$HINGE\_X$')
cbar1.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))


ax3 = plt.subplot(2,1,2, sharex=ax1)
scatterLeftBottom = ax3.scatter(labels[:,5], diffLabels[:,5], s=3, c=labels[:,2], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax3.set_xlabel('$BUFFER\_L$')
ax3.set_xticks([-1., -0.5, 0, 0.5, 1, 1.5, 2.])
ax3.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))

ax3.set_ylabel('$e_{Bl}^u$')
ax3.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))

cbar3 = plt.colorbar(scatterLeftBottom, label='LOCK\_L')
cbar3.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

plt.subplots_adjust(hspace=0.05)

plt.show()

plt.figure(figsize=(18,6))

ax2 = plt.subplot(2,1,1)
scatterRight = plt.scatter(labels[:,4], diffLabels[:,4], s=3, c=labels[:,1], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax2.set_ylabel('$e_{Br}^u$')
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax2.grid()

plt.setp(ax2.get_xticklabels(), visible=False)

cbar2 = plt.colorbar(scatterRight, label='HINGE\_Y')
cbar2.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

ax4 = plt.subplot(2,1,2, sharex=ax2)
scatterRight = plt.scatter(labels[:,4], diffLabels[:,4], s=3, c=labels[:,3], cmap='RdBu_r', vmin=-0.75, vmax=0.75)

ax4.set_xlabel('$BUFFER\_R$')
ax4.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))

ax4.set_ylabel('$e_{Br}^u$')
ax4.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax4.grid()

cbar4 = plt.colorbar(scatterRight, label='LOCK\_R')
cbar4.ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$ mm$'.format(x*1e3)))

plt.subplots_adjust(hspace=0.05)


plt.show()

In [ ]:
print(diffLabels[:,4].min(), diffLabels[:,4].max())
print(diffLabels[:,5].min(), diffLabels[:,5].max())

# Noisy VS. predicted

In [ ]:
diffLabels = predLabelsNoise - predLabels

discritizedX = np.zeros((20,))
discritizedY = np.zeros((20,))
for i in range(1400):
    targetX = np.floor(10 * labels[i,0]).astype(np.int64)
    discritizedX[targetX+10] = max( discritizedX[targetX+10], abs(diffLabels[i,0]) )
    
    targetY = np.floor(10 * labels[i,1]).astype(np.int64)
    discritizedY[targetY+10] = max( discritizedY[targetY+10], abs(diffLabels[i,1]) )
    
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.01


rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.2, height]

fig = plt.figure(figsize=(18,8))

ax_scatter = plt.axes(rect_scatter)
ax_scatter.tick_params(direction='in', top=True, right=True)
ax_histx = plt.axes(rect_histx)
ax_histx.tick_params(direction='in', labelbottom=False)
ax_histy = plt.axes(rect_histy)
ax_histy.tick_params(direction='in', labelleft=False)

magnitudeDiff = np.sqrt(diffLabels[:,0]**2 + diffLabels[:,1]**2)

# the scatter plot:
ax_scatter.scatter(labels[:,0], labels[:,1], s=magnitudeDiff*2e3, alpha=0.8, edgecolors='none', color='tab:blue')

ax_histx.bar(np.arange(20)+0.5, discritizedX)

ax_histy.barh(np.arange(20)+0.5, discritizedY)


ax_scatter.set_xlabel('HINGE\_X')
ax_scatter.set_xticks(np.arange(-1, 1.5, 0.5))
ax_scatter.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))

ax_scatter.set_ylabel('HINGE\_Y')
ax_scatter.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.1fmm'))
ax_scatter.grid()

ax_histy.set_xlabel('$e_H^u|_y$')
ax_histy.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
plt.setp( ax_histy.xaxis.get_majorticklabels(), ha="left" )
ax_histy.grid()

ax_histx.set_ylabel('$e_H^u|_x$')
ax_histx.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x,pos: '{:.0f}$\mu m$'.format(x*1e3)))
ax_histx.grid()

ax_histy.set_ylim(-1, 21)
ax_histx.set_xlim(-1, 21)

handles, l = sca.legend_elements(
    prop="sizes",
    num=5,
    color='tab:blue',
    markeredgewidth=0,
    alpha=0.8,
    fmt="{x:.0f}$\\mu m$",
    func=lambda s: s/2*10
)
legend2 = ax_scatter.legend(
    handles, 
    l, 
    loc="upper right", 
    title="Magnitude", 
    bbox_to_anchor=(1.335, 1.345), 
    ncol=2,
    fontsize=5,
    labelspacing=0.5,
    frameon=True,
    fancybox=False,
    handletextpad=0.,
    columnspacing=0.5
)

plt.show()